### Quickstart - Dicas básicas para entendimento e início de desenvolvimento

**Importação das bibliotecas que serão utilizadas**

In [9]:
import epmwebapi as epm # biblioteca para interação com o EPM
import datetime
import pytz
import getpass
import numpy as np

**Conectando ao EPM Web Server**

In [11]:
# Método para não expor usuário e senha do EPM no código fonte
user = input('EPM user: ')
password = getpass.getpass("EPM password: ")

# Crie o objeto de conexão informando os endereços do EPM Web Server (Authentication Port e Web API Port), usuário e senha.
connection = epm.EpmConnection('https://localhost:44340', 'https://localhost:44339', user, password)

EPM user:  sa
EPM password:  ········


**Definindo a BasicVariable utilizada nos exemplos a seguir**

In [17]:
# Substitua o nome a variável pela sua
bvname = 'MemoriaEpmServer'
bv = connection.getDataObjects([bvname])

**Definindo o período de consulta a ser utilizado nos exemplos a seguir (com fuso-horário)**

In [18]:
# Definindo fuso-horário a ser utilizado
tz_sp = pytz.timezone('America/Sao_Paulo')

# Definindo intervalo de consulta com base no fuso-horário
end_date = datetime.datetime.now(tz_sp)
ini_date= end_date - datetime.timedelta(hours=1)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

**Leitura de dados históricos brutos (Raw) a BasicVariable informada**

In [19]:
data = bv[bvname].historyReadRaw(queryPeriod)

print(data)

[(104444., datetime.datetime(2025, 10, 8, 16, 28, 48, 462000, tzinfo=datetime.timezone.utc), 0)
 (102292., datetime.datetime(2025, 10, 8, 16, 29, 8, 518000, tzinfo=datetime.timezone.utc), 0)
 (103332., datetime.datetime(2025, 10, 8, 16, 29, 38, 623000, tzinfo=datetime.timezone.utc), 0)
 (103792., datetime.datetime(2025, 10, 8, 16, 30, 8, 702000, tzinfo=datetime.timezone.utc), 0)
 (101992., datetime.datetime(2025, 10, 8, 16, 30, 18, 748000, tzinfo=datetime.timezone.utc), 0)
 (103220., datetime.datetime(2025, 10, 8, 16, 31, 8, 912000, tzinfo=datetime.timezone.utc), 0)
 (107328., datetime.datetime(2025, 10, 8, 16, 31, 18, 953000, tzinfo=datetime.timezone.utc), 0)
 (109924., datetime.datetime(2025, 10, 8, 16, 31, 28, 985000, tzinfo=datetime.timezone.utc), 0)
 (107848., datetime.datetime(2025, 10, 8, 16, 31, 39, 10000, tzinfo=datetime.timezone.utc), 0)
 (110464., datetime.datetime(2025, 10, 8, 16, 38, 10, 205000, tzinfo=datetime.timezone.utc), 0)
 (108640., datetime.datetime(2025, 10, 8, 16

**Leitura usando agregação de dados (Baseados na norma OPC-UA) da BasicVariable informada**  
Para informações sobre os tipos de agregações, consulte o Apêndice A do Guia do Usuário: https://github.com/elipsesoftware/epmprocessor/blob/master/guiadousuario/Agregacoes.md

In [20]:
# Definindo o intervalo de processamento/amostragem (sample interval) 
processInterval = datetime.timedelta(minutes=10)
aggDetails = epm.AggregateDetails(processInterval, epm.AggregateType.Interpolative)

data = bv[bvname].historyReadAggregate(aggDetails, queryPeriod)

print(data)

[(104654.21 , datetime.datetime(2025, 10, 8, 16, 26, 36, 616657, tzinfo=datetime.timezone.utc), 1026)
 (109838.16 , datetime.datetime(2025, 10, 8, 16, 36, 36, 616657, tzinfo=datetime.timezone.utc), 1026)
 (111534.48 , datetime.datetime(2025, 10, 8, 16, 46, 36, 616657, tzinfo=datetime.timezone.utc), 1026)
 (113590.164, datetime.datetime(2025, 10, 8, 16, 56, 36, 616657, tzinfo=datetime.timezone.utc), 1026)
 (116284.5  , datetime.datetime(2025, 10, 8, 17, 6, 36, 616657, tzinfo=datetime.timezone.utc), 1026)
 (110044.66 , datetime.datetime(2025, 10, 8, 17, 16, 36, 616657, tzinfo=datetime.timezone.utc), 1026)]


**Escrita de dados históricos na BasicVariable informada**

In [21]:
# Valores de teste para serem escritos
newvalues = [50,60,30,40,10]

timebase = datetime.datetime.now(tz_sp)
newdates = np.array([timebase + datetime.timedelta(hours=i) for i in range(5)])

# Criando objeto ndarray no formato suportado pelo EPM.
desc = np.dtype([('Value', '>f8'), ('Timestamp', 'object'), ('Quality', 'object')])
datatemp = np.empty(len(newvalues), dtype=desc)

# Looping para preencher os dados.
i=0
while i < len(newvalues):
    datatemp['Value'][i] = newvalues[i]
    datatemp['Timestamp'][i] = newdates[i]
    datatemp['Quality'][i] = 0
    i = i+1

# Gravação de dados no EPM.
bv[bvname].historyUpdate(datatemp)

**Escrita de dados de tempo real em uma BasicVariable**

In [26]:
date = datetime.datetime.now(tz_sp)
value = 100
quality = 0 # O valor 0 (zero) equivale à qualidade boa (Good) em OPC-UA.

bv[bvname].write(value, date, quality)

**Leitura do valor de tempo real da variável**

In [29]:
data = bv[bvname].read()

print(data.value)
print(data.timestamp)

100
2025-10-08 18:03:22.140723+00:00


**Escrevendo Anotações**

A estrutura de dados do EPM permite que se escrevam anotações nas variáveis. A anotação consiste em mensagem, estampa de tempo e usuário. Nesse exemplo vamos escrever uma anotação no valor máximo da variável no período consultado.

In [35]:
data = bv[bvname].historyReadRaw(queryPeriod)

max_value = data['Value'].max()

max_index = np.argwhere(data['Value'] == max_value)

print(max_index)

print(data['Timestamp'][max_index][0])

# Escrevendo anotação
message = 'valor maximo encontrado no periodo'
timestamp = data['Timestamp'][max_index][0][0]
bv[bvname].writeAnnotation(timestamp, message)

[[37]]
[datetime.datetime(2025, 10, 8, 17, 3, 55, 115000, tzinfo=datetime.timezone.utc)]


**Consultando anotações**

In [36]:
annotations = bv[bvname].readAnnotations(ini_date,end_date)
print(annotations)

[(datetime.datetime(2025, 10, 8, 14, 3, 55, 115000, tzinfo=tzoffset(None, -10800)), 'sa', 'valor maximo encontrado no periodo')]


**Fechando a conexão**

É importante fechar a conexão para liberá-la para outros possíveis clients.

In [37]:
connection.close()